# Intro: coupon collecting problem
In probability theory, the famous coupon collecting problem is stated as follows: There are n different kinds of coupons, and each time you buy a bag of cereal you can get any 1 coupon with probability $1/n$. What's the expected number of bags of cereal you need to buy to collect all n kinds of coupons? 

The solution is quite simple. Suppose $T_i$ is the number of bags you need to buy to collect i different coupons when you already have i-1 different coupons, $p_i$ is the probability that you collect the new type of coupon when you already have $i-1$ different coupons. Then $p_i = \frac{n-i+1}{n}$. Since $T_i$ is geometrically distributed with probability $p_i$, we have the expected number of bags computed as $E[T] = E[T_1] + E[T_2] + \dots + E[T_n] = \sum_{i=1}^n \frac{1}{p_i} = n\sum_{i=1}^n \frac{1}{i}$. When $n \rightarrow \infty$, $E[T]$ is close to $n\log(n)$.

# Problem generalization
In classic coupon collecting problem, we assumed each coupon can be collected with equal probability. However, in practice, this is usually not the case. There are some coupons easier to collect while others are more difficult. What if probabilities are different? Specifically, we are interested in the following problem: <br>
<blockquote> There are n different coupons. Suppose each draw has probability $p_i$ to get coupon $i$ ($\sum_{i=1}^n p_i <= 1$). What's the expected number of draws to collect all the coupons? </blockquote>

## Let's start with n=2 coupons 

Suppose $T$ is the number of draws to collect both of the coupons, $T_i$ is the number of draws to collect coupon $i$, then using conditional probabilities and the linearity of expected value, we have
$E[T] = E[T_1] + P(\text{coupon 2 is collected after coupon 1})E[T_2]$.
Clearly, $E[T_i] = \frac{1}{p_i}$, we only need to find the probability that coupon 2 is collected after coupon 1. 
That probability can be calculated via $p_1 + (1-p_1-p_2)p_1 + (1-p_1-p_2)^2p_1 + \cdots = \frac{p_1}{p_1+p_2}$. <br>
Therefore, $E[T] = \frac{1}{p_1} + \frac{p_1}{p_1+p_2} \frac{1}{p_2} = \frac{p_1^2+p_2^2+p_1p_2}{p_1p_2(p_1+p_2)}$

We wrote the following code to verify this conclusion

In [6]:
import numpy as np

In [7]:
p1 = 0.1
p2 = 0.2

In [8]:
def count(x):
    i = 0
    j = 0
    c = []
    while i < len(x) and j<len(x):
        j = i
        d = {}
        while j < len(x):
            if x[j]>0:
                d[x[j]] = True
            if len(d)==2:
                c.append(j-i+1)
                i = j+1
                break
            else:
                j = j+1
    return c

In [31]:
def verify(p1, p2, sample_size=1000000):
    x = np.random.choice([1, 2, 0], sample_size, p=[p1, p2, 1-p1-p2])
    a = count(x)
    print(f'''numerical results: {round(np.array(a).mean(),2)}''')
    print(f'''theorical results: {round((p1**2+p2**2+p1*p2)/p1/p2/(p1+p2), 2)}''')
    

In [32]:
verify(0.1, 0.05)

numerical results: 23.31
theorical results: 23.33


# When n>2
Using the above code, we can easily compute the numerical results for any cases. 

In [64]:
def expected_num_coupons(p, sample_size=1000000):
    def _count(n, x):
        '''
            Given array x of simulated coupon collecting process and number of coupons n in total, find number of draws to collect n coupons in total. 
        '''
        i = 0
        j = 0
        c = []
        while i < len(x) and j<len(x):
            j = i
            d = {}
            while j < len(x):
                if x[j]>0:
                    d[x[j]] = True
                if len(d)==n:
                    c.append(j-i+1)
                    i = j+1
                    break
                else:
                    j = j+1
        return c
    
    n = len(p)
    x = np.random.choice(list(range(1,n+1))+[0], sample_size, p=p+[1-np.array(p).sum()])
    a = np.array(_count(n, x))
    return [a.mean(), a.std()]

In [65]:
[me, std] = expected_num_coupons([0.1, 0.2, 0.03])

In [67]:
print(me, std)

36.05856050771672 31.352230962583036


In [68]:
[me, std] = expected_num_coupons([0.1, 0.2, 0.03, 0.04, 0.05])

In [69]:
print(me, std)

49.8074911590377 31.265872158528143


### Can we derive a mathematical formula like we did to $n=2$? 

Suppose coupons are collected according to a Poisson Process with rate $\lambda=1$. A type $i$ event occur when a type $i$ coupon is collected during the process. 
Let ${N_j(t)}$ denote the number of type $j$ coupons collected by time $t$, then $\{N_j(t), t\geq0\}, j=1, \dots, m$ are independent Poisson processes with rates $\lambda_j = \lambda p_j = p_j$. <br>
Let $X_j$ denote the time of the first event of the $j$-th process, $X = \max_{1\leq j \leq n} X_j$ is the time at which all types of coupons are collected. <br>
$P(X<t) = P(X_i<t, \forall i=1,\dots,m) = \Pi_{i=1}^m P(X_i<t)= \Pi_{i=1}^m (1-e^{-p_i t})$ <br>
Therefore, the expected number of draws to collect all types of coupons is $E[X] = \int_{0}^{\infty} P(X>t)dt = \int_0^{\infty} \{1- \Pi_{i=1}^m (1-e^{-p_i t})\}dt$ <br>
To simplify the above formula, we note that $\int_0^{\infty} e^{-\alpha t}dt = -\frac{1}{\alpha}$ when $\alpha<0$. Therefore, 
$E[X] = \sum_{i} \frac{1}{p_i} - \sum_{(i,j)} \frac{1}{p_i+p_j} + \sum_{(i,j,k)}\frac{1}{p_i+p_j+p_k} - \dots$, or more formally,
<blockquote> 
    $E[X] = \sum_{k=1}^n \sum_{i_1, i_2, \dots, i_k} (-1)^{(k-1)} \frac{1}{p_{i_1}+\cdots+p_{i_k}}$
    </blockquote>
When $n=2$, it reduces to $\frac{1}{p_1} + \frac{1}{p_2} - \frac{1}{p_1+p_2}$.
We can write code to compute this number numerically. But it requires $O(2^n)$ time complexity to actually compute this number.

1/0.1+1/0.2+1/0.03-1/0.13-1/0.23-1/0.3+1/0.33

In [71]:
def recur(p, m, n_ele, cur_s, s):
    if n_ele==m:
        s[0] = s[0]+1/cur_s
    else:
        for i in range(len(p)):
            recur(p[i+1:], m, n_ele+1, cur_s+p[i], s)

In [73]:
res = [0]
recur([0.1, 0.2, 0.03], 2, 0, 0, res)

In [76]:
def cal_expected_num_draws(p):
    n = len(p)
    ans = 0
    for k in range(1,n+1):
        res = [0]
        recur(p, k, 0, 0, res)
        ans = ans + np.power(-1, k-1)*res[0]
    return ans

In [85]:
p=[0.1, 0.2, 0.03, 0.008]
print(cal_expected_num_draws(p))
print(expected_num_coupons(p)[0])

132.34365905936258
132.09180977542934


In [95]:
%%time
n = 24
p = [1/n for i in range(n)]
print(cal_expected_num_draws(p))
np.array([1/i for i in range(1,n+1)]).sum()*n

90.62274739073663
CPU times: user 54.6 s, sys: 119 ms, total: 54.8 s
Wall time: 54.9 s


90.62299626608417

# Open questions
 - How to efficiently compute the mean when there are more than 30 types of coupons?
 - Is there a closed form solution for the variance as well?